In [47]:
import string
import sys
import getopt
import datetime
import xml.etree.ElementTree as ET

In [ ]:
import pickle

In [ ]:
messages = pickle.load( open( "MessageDict.p", "rb" ) )
groups = pickle.load( open( "GroupsDict.p", "rb" ) )

In [ ]:
print(messages['1'])

In [ ]:
line = '8=FIX.4.4<SOH>9=326<SOH>35=D<SOH>453=3<SOH>448=DEU<SOH>447=B<SOH>452=1<SOH>802=1<SOH>523=A1<SOH>803=10<SOH>448=104317<SOH>447=H<SOH>452=83<SOH>448=GSI<SOH>447=B<SOH>452=4<SOH>2376=23<SOH>802=1<SOH>523=C3<SOH>803=10<SOH>'

delim = '<SOH>'
strt_idx = line.find('8=FIX')
line = line[strt_idx:]
columns = line.split(delim)

for col in columns:
    values = col.split('=')
    if len(values) > 1: # This ignores all \n
        key = int(values[0])
        if key==35:
            msg_type = values[1]

            
# Make a class with this method
def return_level_change():
    
    
    
    
    

# Main application

Importing xml

In [5]:
import xmltodict
f = open("OrchestraEP257.xml", "r", encoding="utf8")
# convert to dictionary
fixindict = xmltodict.parse(f.read())
f.close()

Test cases!

2 - multiple fields, 1 group, no components

12 - multiple groups

13 - multiple groups, 1 component

15 - 1 group, multiple components

## Field Class

In [19]:
# This is all fields. You can use for field in fields
fields = fixindict['fixr:repository']['fixr:fields']['fixr:field']

print((fields[0]))

OrderedDict([('@added', 'FIX.2.7'), ('@id', '1'), ('@name', 'Account'), ('@type', 'String'), ('@abbrName', 'Acct'), ('fixr:annotation', OrderedDict([('fixr:documentation', OrderedDict([('@purpose', 'SYNOPSIS'), ('#text', 'Account mnemonic as agreed between buy and sell sides, e.g. broker and institution or investor/intermediary and fund manager.')]))]))])


In [41]:
class MyField():

    '''
    Usage: Call field class on individual fields to parse field data
    In the dictionary, key is id and value is a list containing name, type and abbr
    '''

    def __init__(self, field):
        self.id = field['@id']
        self.name = field['@name']
        self.type = field['@type']
        try:
            self.abbrname = field['@abbrName']
        except:
            self.abbrname = 'abbrName not found'

    def dictify(self, field_dict):
        '''Appends key "id" with a list of name, type and abbrname'''

        field_dict[self.id] = [self.name, self.type, self.abbrname]


a = MyField(fields[0])
print(a.id)
print(a.name)

1
Account


In [45]:
# Field class in action

field_dict = {}
for i in range(len(fields)):
    temp_field = MyField(fields[i])
    temp_field.dictify(field_dict)

print(len(field_dict))
field_dict['3'][0]

5936


'AdvRefID'

## Message class

This class would contain all outputs for 35

In [56]:
messages = fixindict['fixr:repository']['fixr:messages']['fixr:message']

print((messages[0]['fixr:structure']))

OrderedDict([('fixr:componentRef', [OrderedDict([('@id', '1024'), ('@presence', 'required'), ('@added', 'FIX.2.7'), ('fixr:annotation', OrderedDict([('fixr:documentation', 'MsgType = 0')]))]), OrderedDict([('@id', '1025'), ('@presence', 'required'), ('@added', 'FIX.2.7'), ('fixr:annotation', OrderedDict([('fixr:documentation', None)]))])]), ('fixr:fieldRef', OrderedDict([('@id', '112'), ('@added', 'FIX.4.0'), ('fixr:annotation', OrderedDict([('fixr:documentation', 'Required when the heartbeat is the result of a Test Request message.')]))]))])


In [66]:
# Testing structure in messages
# Message 6 is good for testing - name IOI
struct = messages[6]['fixr:structure']

comp = struct['fixr:componentRef']
fl = struct['fixr:fieldRef']
gp = struct['fixr:groupRef']

print(len(comp))
print(len(fl))
print(len(gp))

for c in comp:
    print(c['@id'])


9
17
9
1024
1057
1003
1004
1002
1011
1018
1022
1025


In [109]:
# Testing structure in messages
# Message 0 is good for testing single fields
struct = messages[0]['fixr:structure']

comp = struct['fixr:componentRef']
fl = struct['fixr:fieldRef']

print(len(comp))
print(len(fl))




2
3


In [99]:
class MyMessage():
    
    def __init__(self, msg):
        self.id = msg['@id']
        self.name = msg['@name']
        self.msgtype = msg['@msgType']
        
        structure = msg['fixr:structure']
        self.components = []
        self.fields = []
        self.groups = []
        # Components
        try:
            comps = structure['fixr:componentRef']
            for c in comps:
                self.components.append(c['@id'])
        except:
            try:
                comp = structure['fixr:componentRef']
                self.components.append(comp['@id'])
            except:
                pass
        # Fields
        try:
            fields = structure['fixr:fieldRef']
            for f in fields:
                self.fields.append(f['@id'])
        except:
            try:
                field = structure['fixr:fieldRef']
                self.fields.append(field['@id'])
            except:
                pass
        # Groups
        try:
            groups = structure['fixr:groupRef']
            for g in groups:
                self.groups.append(g['@id'])
        except:
            try:
                group = structure['fixr:groupRef']
                self.fields.append(group['@id'])
            except:
                pass
    
    
    def dictify(self, dictionary):
        dictionary[self.msgtype] = [self.id, self.name, self.components, self.fields, self.groups]
    
m = MyMessage(messages[6])
# print(m.id)
msgdicttrial = {}
m.dictify(msgdicttrial)
print(msgdicttrial)

{'6': ['7', 'IOI', ['1024', '1057', '1003', '1004', '1002', '1011', '1018', '1022', '1025'], ['23', '28', '26', '54', '854', '27', '15', '423', '44', '62', '25', '130', '58', '354', '355', '60', '149'], ['1012', '2066', '1066', '1019', '2020', '1074', '2024', '2054', '2252']]}


In [100]:
msg_dict = {}
for i in range(len(messages)):
    temp_field = MyMessage(messages[i])
    temp_field.dictify(msg_dict)

print((msg_dict))
#msg_dict['D']

{'0': ['1', 'Heartbeat', ['1024', '1025'], ['112'], []], '1': ['2', 'TestRequest', ['1024', '1025'], ['112'], []], '2': ['3', 'ResendRequest', ['1024', '1025'], ['7', '16'], []], '3': ['4', 'Reject', ['1024', '1025'], ['45', '371', '372', '1130', '1406', '1131', '373', '58', '354', '355'], []], '4': ['5', 'SequenceReset', ['1024', '1025'], ['123', '36'], []], '5': ['6', 'Logout', ['1024', '1025'], ['1409', '58', '354', '355'], []], '6': ['7', 'IOI', ['1024', '1057', '1003', '1004', '1002', '1011', '1018', '1022', '1025'], ['23', '28', '26', '54', '854', '27', '15', '423', '44', '62', '25', '130', '58', '354', '355', '60', '149'], ['1012', '2066', '1066', '1019', '2020', '1074', '2024', '2054', '2252']], '7': ['8', 'Advertisement', ['1024', '1003', '1004', '1002', '1025'], ['2', '5', '3', '4', '53', '854', '44', '15', '75', '60', '58', '354', '355', '149', '30', '336', '625'], ['2019', '2066', '1066', '2054']], '8': ['9', 'ExecutionReport', ['1024', '1057', '1003', '1002', '1011', '1030

## Component class

In [103]:
components = fixindict['fixr:repository']['fixr:components']['fixr:component']

print((components[4]))

OrderedDict([('@name', 'InstrumentExtension'), ('@id', '1004'), ('@category', 'Common'), ('@added', 'FIX.4.4'), ('@abbrName', 'InstrmtExt'), ('fixr:fieldRef', [OrderedDict([('@id', '668'), ('@added', 'FIX.4.4'), ('fixr:annotation', OrderedDict([('fixr:documentation', 'Identifies the form of delivery.')]))]), OrderedDict([('@id', '869'), ('@added', 'FIX.4.4'), ('fixr:annotation', OrderedDict([('fixr:documentation', 'Percent at risk due to lowest possible call.')]))]), OrderedDict([('@id', '2736'), ('@added', 'FIX.5.0SP2'), ('@addedEP', '235'), ('fixr:annotation', OrderedDict([('fixr:documentation', None)]))]), OrderedDict([('@id', '2738'), ('@added', 'FIX.5.0SP2'), ('@addedEP', '235'), ('fixr:annotation', OrderedDict([('fixr:documentation', None)]))])]), ('fixr:groupRef', [OrderedDict([('@id', '2074'), ('@added', 'FIX.4.4'), ('fixr:annotation', OrderedDict([('fixr:documentation', 'Number of repeating InstrAttrib group entries.')]))]), OrderedDict([('@id', '2262'), ('@added', 'FIX.5.0SP2

In [115]:
class MyComponent():
    
    def __init__(self, component):
        self.id = component['@id']
        self.name = component['@name']
        
        self.components = []
        self.fields = []
        self.groups = []
        # Components
        try:
            comps = component['fixr:componentRef']
            for c in comps:
                self.components.append(c['@id'])
        except:
            try:
                comp = component['fixr:componentRef']
                self.components.append(comp['@id'])
            except:
                pass
        # Fields
        try:
            fields = component['fixr:fieldRef']
            for f in fields:
                self.fields.append(f['@id'])
        except:
            try:
                field = component['fixr:fieldRef']
                self.fields.append(field['@id'])
            except:
                pass
        # Groups
        try:
            groups = component['fixr:groupRef']
            for g in groups:
                self.groups.append(g['@id'])
        except:
            try:
                group = component['fixr:groupRef']
                self.groups.append(group['@id'])
            except:
                pass
    
    
    def dictify(self, dictionary):
        dictionary[self.id] = [self.name, self.components, self.fields, self.groups]

c = MyComponent(components[4])
print(c.id)
compdicttrial = {}
c.dictify(compdicttrial)
print(compdicttrial)

1004
{'1004': ['InstrumentExtension', ['2264'], ['668', '869', '2736', '2738'], ['2074', '2262', '2263']]}


In [116]:
comp_dict = {}
for i in range(len(components)):
    temp_field = MyComponent(components[i])
    temp_field.dictify(comp_dict)

print((comp_dict))
#msg_dict['D']

{'1000': ['CommissionData', [], ['12', '13', '479', '1233', '1238', '497'], []], '1001': ['DiscretionInstructions', [], ['388', '389', '841', '842', '843', '844', '846'], []], '1002': ['FinancingDetails', [], ['913', '914', '1961', '915', '918', '1962', '1963', '1964', '1965', '1966', '1967', '1968', '1969', '1970', '1513', '1525', '1527', '788', '916', '917', '919', '898'], ['4003', '4005', '4004']], '1003': ['Instrument', ['1060', '4085', '4175', '4158', '4162'], ['55', '65', '48', '22', '460', '1227', '1151', '461', '167', '762', '200', '541', '1079', '966', '1049', '965', '224', '1449', '1450', '1451', '1452', '1457', '1458', '1739', '2210', '1938', '1939', '1940', '2735', '1941', '1575', '1942', '1943', '1944', '1945', '1946', '1947', '1948', '1949', '1950', '2879', '1951', '1952', '1953', '1954', '1955', '1956', '1957', '1958', '1959', '1960', '1577', '1580', '1581', '1678', '1697', '225', '239', '226', '227', '228', '255', '543', '470', '471', '472', '240', '202', '2578', '2577'

## Group class

In [108]:
groups = fixindict['fixr:repository']['fixr:groups']['fixr:group']

print((groups[4]))

OrderedDict([('@id', '1015'), ('@added', 'FIX.4.4'), ('@name', 'PositionQty'), ('@category', 'Common'), ('@abbrName', 'Qty'), ('fixr:numInGroup', OrderedDict([('@id', '702'), ('fixr:annotation', OrderedDict([('fixr:documentation', None)]))])), ('fixr:fieldRef', [OrderedDict([('@id', '703'), ('@added', 'FIX.4.4'), ('fixr:annotation', OrderedDict([('fixr:documentation', 'Required if NoPositions > 1')]))]), OrderedDict([('@id', '704'), ('@added', 'FIX.4.4'), ('fixr:annotation', OrderedDict([('fixr:documentation', None)]))]), OrderedDict([('@id', '705'), ('@added', 'FIX.4.4'), ('fixr:annotation', OrderedDict([('fixr:documentation', None)]))]), OrderedDict([('@id', '1654'), ('@added', 'FIX.5.0SP2'), ('@addedEP', '103'), ('fixr:annotation', OrderedDict([('fixr:documentation', 'Short quantity that is considered covered, e.g. used for short option position')]))]), OrderedDict([('@id', '706'), ('@added', 'FIX.4.4'), ('fixr:annotation', OrderedDict([('fixr:documentation', None)]))]), OrderedDict

In [123]:
class MyGroup():
    
    def __init__(self, group):
        # id
        self.id = group['@id']
        self.name = group['@name']

        # NuminGroup
        self.numingroup = group['fixr:numInGroup']['@id']

        self.components = []
        self.fields = []
        self.groups = []
        # Components in group
        try:
            comps = group['fixr:componentRef']
            for c in comps:
                self.components.append(c['@id'])
        except:
            try:
                comp = group['fixr:componentRef']
                self.components.append(comp['@id'])
            except:
                pass
        # Fields in group
        try:
            fields = group['fixr:fieldRef']
            for f in fields:
                self.fields.append(f['@id'])
        except:
            try:
                field = group['fixr:fieldRef']
                self.fields.append(field['@id'])
            except:
                pass
        # Groups in group
        try:
            groups = group['fixr:groupRef']
            for g in groups:
                self.groups.append(g['@id'])
        except:
            try:
                grp = group['fixr:groupRef']
                self.groups.append(grp['@id'])
            except:
                pass
            
    def dictify(self, dictionary):
        dictionary[self.id] = [self.name, self.components, self.fields, self.groups]
        
    # def in_current_level(self,itm):
    #     if itm in self.fieldref or itm in self.groupref or itm in self.componentref:
    #         level_change = 0
    #         return level_change
    #     elif itm in :
    #         pass
    #     else:
    #         pass
    #         # Here, I may need to define new group class, check those components
    #         # This entire function may need to be moved to groups class, or even fixr class
    

g = MyGroup(groups[17])
print(g.id)
groupdicttrial = {}
g.dictify(groupdicttrial)
print(groupdicttrial)


2007
{'2007': ['ClrInstGrp', [], ['577'], []]}


Test cases! <br />
2 - multiple fields, 1 group, no components <br />
12 - multiple groups (2 groups) <br />
13 - multiple groups, 1 component <br />
15 - 1 group, multiple components <br />

In [125]:
group_dict = {}
for i in range(len(groups)):
    temp_field = MyGroup(groups[i])
    temp_field.dictify(group_dict)

print((group_dict['2010']))


['CompIDStatGrp', [], ['930', '931', '283', '284', '928', '929'], []]


# OrchestraXML class

In [142]:
import xmltodict

class OrchestraXML():
    def __init__(self, xml):
        f = open(xml, "r", encoding="utf8")
        fixindict = xmltodict.parse(f.read())
        f.close()
        
        fields = fixindict['fixr:repository']['fixr:fields']['fixr:field']
        self.field_dict = {}
        for i in range(len(fields)):
            temp_field = MyField(fields[i])
            temp_field.dictify(self.field_dict)
            
        messages = fixindict['fixr:repository']['fixr:messages']['fixr:message']
        self.msg_dict = {}
        for i in range(len(messages)):
            temp_field = MyMessage(messages[i])
            temp_field.dictify(self.msg_dict)
            
        components = fixindict['fixr:repository']['fixr:components']['fixr:component']
        self.comp_dict = {}
        for i in range(len(components)):
            temp_field = MyComponent(components[i])
            temp_field.dictify(self.comp_dict)
        
        groups = fixindict['fixr:repository']['fixr:groups']['fixr:group']
        self.group_dict = {}
        for i in range(len(groups)):
            temp_field = MyGroup(groups[i])
            temp_field.dictify(self.group_dict)
        
        
fix = OrchestraXML("OrchestraEP257.xml") 

# fix.field_dict
# fix.msg_dict
# fix.comp_dict
# fix.group_dict






{'1007': ['LegStipulations', [], ['688', '689'], []],
 '1008': ['NestedParties', [], ['524', '525', '538', '2384'], ['2078']],
 '1012': ['Parties', [], ['448', '447', '452', '2376'], ['2077']],
 '1014': ['PositionAmountData',
  [],
  ['707',
   '708',
   '2096',
   '1055',
   '2097',
   '2098',
   '1585',
   '2099',
   '2100',
   '2876',
   '2877'],
  []],
 '1015': ['PositionQty',
  [],
  ['703', '704', '705', '1654', '706', '976', '1836', '1835'],
  ['1008']],
 '1017': ['SettlParties', [], ['782', '783', '784', '2389'], ['2076']],
 '1019': ['Stipulations', [], ['233', '234'], []],
 '1020': ['TrdRegTimestamps',
  [],
  ['769',
   '770',
   '771',
   '2839',
   '1033',
   '1034',
   '1035',
   '1727',
   '2831',
   '2832',
   '2833',
   '2834'],
  []],
 '1023': ['UnderlyingStipulations', [], ['888', '889'], []],
 '1009': ['NestedParties2', [], ['757', '758', '759', '2381'], ['2079']],
 '1010': ['NestedParties3', [], ['949', '950', '951', '2382'], ['2080']],
 '2001': ['AffectedOrdGrp', [

# Old

## Original xml parser

In [130]:
import xml.etree.ElementTree as ET
def parse_xml(file, dict, msg_dict, field_enum_dict):
    print ("Parsing file: %s" % file)
    tree = ET.parse(file)
    #print(tree)
    root = tree.getroot()
    #print(root)
    
    # This goes into the xml file, to <fields> and adds all fields to fields.
    fields = root.findall("fields")[0].findall('field')
    
    #print(fields)
    for field in fields:
        #print(field.attrib['number']) # This is the number associated with each field
        #print(field.attrib['name']) # This is the name of each field
        
        # So this adds number to dict with name as value
        dict[int(field.attrib['number'])] = field.attrib['name']
        
        # If some fields have values, they are saved here as a list
        values = field.findall("value")
        #print(values)
        
        # This starts a new dictionary
        enum_dict = {}
        for value in values:
            # This saved enum as key, desc as value from values
            # Also, this is multiple dicts? Most likely
            enum_dict[value.attrib['enum']] = value.attrib['description']
        #print((enum_dict))
        if len(enum_dict) > 0:
            #print(enum_dict)
            # Adds sub dict in dict for items with more details
            field_enum_dict[int(field.attrib['number'])] = enum_dict
    
    # This takes in <messages>, just as we took <fields> before
    msgs = root.findall("messages")[0].findall('message')
    #print(msgs)
    for msg in msgs:
        msg_dict[msg.attrib['msgtype']] = msg.attrib['name']
        
dict = {} # Has LHS names
msg_dict = {} # Has Message names, only for MessageType(35) 
field_enum_dict = {}


xml_file1 = 'FIX50SP2.xml'
xml_file2 = 'FIXT11.xml'
parse_xml(xml_file1, dict, msg_dict, field_enum_dict)
# parse_xml(xml_file2, dict, msg_dict, field_enum_dict)


print((dict))
#print((msg_dict))
#print((field_enum_dict))


Parsing file: FIX50SP2.xml
{1: 'Account', 2: 'AdvId', 3: 'AdvRefID', 4: 'AdvSide', 5: 'AdvTransType', 6: 'AvgPx', 7: 'BeginSeqNo', 8: 'BeginString', 9: 'BodyLength', 10: 'CheckSum', 11: 'ClOrdID', 12: 'Commission', 13: 'CommType', 14: 'CumQty', 15: 'Currency', 16: 'EndSeqNo', 17: 'ExecID', 18: 'ExecInst', 19: 'ExecRefID', 21: 'HandlInst', 22: 'SecurityIDSource', 23: 'IOIID', 25: 'IOIQltyInd', 26: 'IOIRefID', 27: 'IOIQty', 28: 'IOITransType', 29: 'LastCapacity', 30: 'LastMkt', 31: 'LastPx', 32: 'LastQty', 33: 'NoLinesOfText', 34: 'MsgSeqNum', 35: 'MsgType', 36: 'NewSeqNo', 37: 'OrderID', 38: 'OrderQty', 39: 'OrdStatus', 40: 'OrdType', 41: 'OrigClOrdID', 42: 'OrigTime', 43: 'PossDupFlag', 44: 'Price', 45: 'RefSeqNum', 48: 'SecurityID', 49: 'SenderCompID', 50: 'SenderSubID', 52: 'SendingTime', 53: 'Quantity', 54: 'Side', 55: 'Symbol', 56: 'TargetCompID', 57: 'TargetSubID', 58: 'Text', 59: 'TimeInForce', 60: 'TransactTime', 61: 'Urgency', 62: 'ValidUntilTime', 63: 'SettlType', 64: 'SettlDa

## My old version of pretty printer - brute force

In [135]:

def prettify(src_file, dest_file, dict, msg_dict, field_enum_dict, parse_enum):
    with open(dest_file, 'w') as fo: # REMOVED , 'wb' from open bracket
        with open(src_file, 'r') as f: # REMOVED , 'rb' from open bracket
            print()
            print(src_file)
            
            # Save all lines in src_file
            lines = f.readlines()
            i=-1
            # Get delimiter
            try:
                delim_idx = lines[0].rfind('10=')
                delim = '<SOH>'
                #delim = lines[0][delim_idx-1]
                print(f'The delimiter is {delim} \n')
            except:
                print('Could not identify delimiter in first line of the log file. \nTrying second line. \n')
                delim_idx = lines[1].rfind('10=')
                delim = lines[1][delim_idx-1]
                print(f'The delimiter is {delim} \n')
            
            for line in lines:
                i+=1
                # Find the start index in a line
                strt_idx = line.find('8=FIX')
                line=line[strt_idx:]
                
                # Get all columns from line with delimiter
                columns = line.split(delim)
                for col in columns:
                    
                    #print(col)
                    values = col.split('=')
                    if len(values) > 1: # This ignores all \n
                        try:
                            key = int(values[0])
                            tab = checksubgroup(key)
                            
                            tag_name = dict[key] # Gets tag name from dict
                            
                            # Getting message names from MessageType
                            if key == 35:
                                try:
                                    msg_name = msg_dict[values[1]]
                                    fo.write(tab+"%s(%d) = %s(%s)\n" % (tag_name, key, msg_name, values[1]))
                                except KeyError:
                                    print("msg_name KeyError: %d, value: %s, line_num: %d: tag 35 do not have %s msg listed" % (key, values[1], i, values[1]))
                                    fo.write(tab+"%s(%d) = %s\n" % (tag_name, key, values[1]))
                            
                            # Getting names from fields
                            elif parse_enum and key in field_enum_dict:
                                enum_dict = field_enum_dict[key]
                                
                                
                                
                                if values[1] in enum_dict:
                                    fo.write(tab+"%s(%d) = %s(%s)\n" % (tag_name, key, enum_dict[values[1]], values[1]))
                                else:
                                    fo.write(tab+"%s(%d) = %s\n" % (tag_name, key, values[1]))
                                    print("enum KeyError: %d, value: %s, line_num: %d: tag %d does not have this enum value in dictionary: %s" % (key, values[1], i, key, values[1]))
                            else:
                                fo.write(tab+"%s(%d) = %s\n" % (tag_name, key, values[1]))
                        except ValueError:
                            print(column)
                            fo.write("%s\n" % col)
                        except KeyError:
                            tab = checksubgroup(values[0])
                            fo.write(tab+"%s = %s\n" % (values[0], values[1]))
                            print("tag_name KeyError: %d, value: %s, line_num: %d" % (key, values[1], i))
                fo.write("\n\n")
            
            
            
                
                

src_log = 'logtest.txt'
#src_log2 = 'delimitertest.log'
#src_log3 = 'FIX.4.2-KOD-BON.messages.current.log'
#src_log4 = 'FIXT.1.1-EXEC-BANZAI.messages.log'




output_log = 'log2.txt'
prettify(src_log, output_log, dict, msg_dict, field_enum_dict, True) #parse_enum is set true
#prettify(src_log2, output_log, dict, msg_dict, field_enum_dict, True) #parse_enum is set true
#prettify(src_log3, output_log, dict, msg_dict, field_enum_dict, True) #parse_enum is set true
#prettify(src_log4, output_log, dict, msg_dict, field_enum_dict, True) #parse_enum is set true



logtest.txt
The delimiter is <SOH> 

tag_name KeyError: 2376, value: 23, line_num: 0
log2.txt


In [132]:
def checksubgroup(vals):
    
    indent1=[448,447,452,802,2376]
    indent2=[523,803]
    indent1str=[str(i) for i in indent1]
    indent2str=[str(i) for i in indent2]
    
    if vals in indent1 or vals in indent1str:
        tab = '\t'
        
    elif vals in indent2 or vals in indent2str:
        tab = '\t\t'
    else:
        tab = ''
    return tab

## Original untouched pretty printer

In [134]:
def prettify(src_file, dest_file, dict, msg_dict, field_enum_dict, parse_enum):
    with open(dest_file, 'wb') as fo: # REMOVED , 'wb' from open bracket
        with open(src_file, 'rb') as f: # REMOVED , 'rb' from open bracket
            line_num = 0
            while True:
                line_num+=1
                s = f.readline()
                if len(s) == 0: break
                columns = s.split(chr(1))
                if len(columns) == 1:
                    columns = s.split('|')
                for column in columns:
                    values = column.split('=')
                    if len(values) > 1:
                        try:
                            key = int(values[0])
                            tag_name = dict[key]
                            if key == 35:
                                try:
                                    msg_name = msg_dict[values[1]]
                                    fo.write("%s(%d)=%s(%s)|" % (tag_name, key, msg_name, values[1]))
                                except KeyError:
                                    print("msg_name KeyError: %d, value: %s, line_num: %d: tag 35 do not have %s msg listed" % (key, values[1], line_num, values[1]))
                                    fo.write("%s(%d)=%s|" % (tag_name, key, values[1]))
                            elif parse_enum and field_enum_dict.has_key(key):
                                enum_dict = field_enum_dict[key]
                                if enum_dict.has_key(values[1]):
                                    fo.write("%s(%d)=%s(%s)|" % (tag_name, key, enum_dict[values[1]], values[1]))
                                else:
                                    fo.write("%s(%d)=%s|" % (tag_name, key, values[1]))
                                    print("enum KeyError: %d, value: %s, line_num: %d: tag %d does not have this enum value in dictionary: %s" % (key, values[1], line_num, key, values[1]))
                            else:
                                fo.write("%s(%d)=%s|" % (tag_name, key, values[1]))
                        except ValueError:
                            fo.write("%s|" % column)
                        except KeyError:
                            fo.write("%s=%s|" % (values[0], values[1]))
                            print("tag_name KeyError: %d, value: %s, line_num: %d" % (key, values[1], line_num))
                fo.write("\n")

src_log = 'log.txt'
output_log = 'log2.txt'
prettify(src_log, output_log, dict, msg_dict, field_enum_dict, True) #parse_enum is set true

TypeError: a bytes-like object is required, not 'str'